In [1]:
import os

from openai import OpenAI, Stream
from openai.types.beta.assistant_stream_event import AssistantStreamEvent

from timestep.config import settings

In [2]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY", settings.openai_api_key),
    # base_url=os.getenv("OPENAI_BASE_URL", settings.openai_base_url),
    base_url="http://127.0.0.1:8080/v1",
)

In [3]:
# https://gorilla.cs.berkeley.edu/blogs/8_berkeley_function_calling_leaderboard.html#prompt
SYSTEM_PROMPT_FOR_CHAT_MODEL = """
You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
If none of the function can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.
"""

USER_MESSAGE_FOR_CHAT_MODEL = "Questions:{user_prompt}\nHere is a list of functions in JSON format that you can invoke:\n{functions}. Should you decide to return the function call(s), NO other text MUST be included."

# https://github.com/BerriAI/litellm/blob/feb8c3c5b439cad22d26eeb49852207e3820665d/litellm/llms/prompt_templates/factory.py#L2443
# Function call template
def function_call_prompt(messages: list, functions: list):
    # function_prompt = """Produce JSON OUTPUT ONLY! Adhere to this format {"name": "function_name", "arguments":{"argument_name": "argument_value"}} The following functions are available to you:"""
    function_prompt = SYSTEM_PROMPT_FOR_CHAT_MODEL + "\nHere is a list of functions in JSON format that you can invoke:\n"

    # function_prompt = "Produce JSON OUTPUT ONLY! Adhere to this format {'tool_calls': [{'name': 'function_name', 'arguments': {'argument_name': 'argument_value'}}]} The following functions are available to you:"

    for function in functions:
        function_prompt += f"""\n{function}\n"""

    # function_prompt += "\nShould you decide to return the function call(s), NO other text MUST be included."

    function_added_to_prompt = False
    for message in messages:
        if "system" in message["role"]:
            message["content"] += f""" {function_prompt}"""
            function_added_to_prompt = True

    if function_added_to_prompt == False:
        messages.append({"role": "system", "content": f"""{function_prompt}"""})
        # messages = [{"role": "system", "content": f"""{function_prompt}"""}] + messages

    return messages

In [4]:
# tools = [
#     {
#         "type": "function",
#         "function": {
#             "name": "get_weather",
#             "strict": True,
#             "parameters": {
#                 "type": "object",
#                 "properties": {
#                     "location": {"type": "string"},
#                     "unit": {"type": "string", "enum": ["c", "f"]},
#                 },
#                 "required": ["location", "unit"],
#                 "additionalProperties": False,
#             },
#         },
#     },
    # {
    #     "type": "function",
    #     "function": {
    #         "name": "get_stock_price",
    #         "strict": True,
    #         "parameters": {
    #             "type": "object",
    #             "properties": {
    #                 "symbol": {"type": "string"},
    #             },
    #             "required": ["symbol"],
    #             "additionalProperties": False,
    #         },
    #     },
    # },
# ]

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

# messages = [
#     # {"role": "system", "content": SYSTEM_PROMPT_FOR_CHAT_MODEL},
#     # {"role": "user", "content": "What's the weather like in Boston today?"},
#     # {"role": "user", "content": USER_MESSAGE_FOR_CHAT_MODEL.format(user_prompt="What's the weather like in Boston today?", functions=tools)},
# ]

functions = [tool["function"] for tool in tools]

messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]

# messages = function_call_prompt(messages, tools)
messages = function_call_prompt(messages, functions)

print(messages)

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    # highlight-start
    # tool_choice="required"
    tool_choice="auto"
    # highlight-end
)

print(completion)

[{'role': 'user', 'content': "What's the weather like in San Francisco, Tokyo, and Paris?"}, {'role': 'system', 'content': "\nYou are an expert in composing functions. You are given a question and a set of possible functions.\nBased on the question, you will need to make one or more function/tool calls to achieve the purpose.\nIf none of the function can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.\n\nHere is a list of functions in JSON format that you can invoke:\n\n{'name': 'get_current_weather', 'description': 'Get the current weather in a given location', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}}, 'required': ['location']}}\n"}]
ChatCompletion(id='chatcmpl-3dcIL4SqLrmGEGIgT7mNzpiYE4gbOA3v', cho

In [5]:
raise Exception("stop")

Exception: stop

In [ ]:
response = client.embeddings.create(
    input="Your text string goes here",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)

In [ ]:
len(response.data[0].embedding)

In [ ]:
assistant = client.beta.assistants.create(
    model="gpt-4o"
)

In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA",
          },
          "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
      },
    }
  }
]

stream: Stream[AssistantStreamEvent] = client.beta.threads.create_and_run(
  thread={
      "messages": [
        {"role": "user", "content": "What is the weather like in San Francisco?"}
      ]
  },
  # assistant_id="asst_abc123",
  assistant_id=assistant.id,
  tools=tools,
  stream=True
)

try:
  for event in stream:
    print(event)

finally:
  client.beta.assistants.delete(assistant.id)
  stream.close()